Command Profile in TVB
----------------------

This is a simple demo showing for to interract with TVB in the command profile.

More examples can be found here:
https://github.com/the-virtual-brain/tvb-framework/tree/master/tvb/interfaces/command/demos

In [1]:
from tvb.basic.profile import TvbProfile
TvbProfile.set_profile(TvbProfile.COMMAND_PROFILE)
from tvb.interfaces.command.lab import *

In [2]:
list_projects()

                    name    id
         Default_Project     1


Create a new project to test with

In [3]:
proj = new_project("sandbox")

If you have the web UI open at the same time, you'll notice there's now a new project named "sandbox"

In [5]:
list_datatypes(proj.id)

                    type              tag    id                              gid         date


Nothing! Let's import a connectivity

In [6]:
import os
import tvb_data
p = os.path.join(os.path.dirname(tvb_data.__file__), 'connectivity/connectivity_66.zip')
import_conn_zip(proj.id, p)

In [7]:
list_datatypes(proj.id)

                    type              tag    id                              gid         date
            Connectivity             None    16 53155d80-36d2-11e6-8c5f-64006a75eba5 2016-06-20 12:32:44.735000


Now let's grab that connectivity from the database. 
IMPORTANT: Make sure to adjust the DT id below to match the value in the id column above

In [8]:
conn, = load_dt(16)
conn

Number of connections,1377
Tract lengths - Mean non zero,82.3333994849
Tract lengths - Var. non zero (connections),2727.07958881
Tract lengths - Max. non zero (connections),238.0
Weights - Variance,0.00265501924904
Tract lengths - Variance,2327.55571837
Number of regions,66
Weights - Min. non zero,3.59450032551e-05
Tract lengths - Mean,26.0268804157
Tract lengths - Min. non zero (connections),7.0
Weights - Maximum,0.512164524459


and run a simulation with it

In [11]:
params = dict(connectivity=conn)
sim_op = fire_simulation(project_id=proj.id, connectivity=conn.gid, simulation_length=100)

We should wait for the simulation to finish, and afterwards, TimeSeries should also be part of this project.

Refresh the Web GUI to see the status of your operation.

In [13]:
list_datatypes(proj.id)

                    type              tag    id                              gid         date
            Connectivity             None    16 53155d80-36d2-11e6-8c5f-64006a75eba5 2016-06-20 12:32:44.735000
         SimulationState             None    17 71bb964f-36d2-11e6-a200-64006a75eba5 2016-06-20 12:33:36.091000
        TimeSeriesRegion             None    18 71bad300-36d2-11e6-a55f-64006a75eba5 2016-06-20 12:33:38.256000
         SimulationState             None    19 75c70c70-36d2-11e6-9b2c-64006a75eba5 2016-06-20 12:33:42.898000
        TimeSeriesRegion             None    20 75c4e98f-36d2-11e6-b6ed-64006a75eba5 2016-06-20 12:33:45.309000


You can re-evaluate this cell multiple times while it's running to see how the results gradually show up as the simulation finishes.